In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'E:\Work_files\Twitter sentimnt analysis\csv file\shivsena_bjp.csv')

In [3]:
df.head()

,timestamp,tweet_text,username,all_hashtags,followers_count
0,2019-11-26 10:37:56,So as per mananiya bhavi mukhyamantri Aditya T...,amanporwal75,"[u'ShivSena', u'SanjayRaut', u'UddhavThackeray...",46
1,2019-11-26 10:25:32,@Dev_Fadnavis Sir these 4 days you made #Udha...,chikusrikanth,"[u'UdhavThackeray', u'adityathackeray', u'fadn...",531
2,2019-11-26 09:59:06,@PoulomiMSaha @sanjayraut @AjitPawarSpeaks @Of...,Friendship24x7,"[u'UddhavThackeray', u'AdityaThackeray', u'PCh...",24
3,2019-11-26 09:50:45,I wanna see @AUThackeray as our Education Mini...,Yuvraaaj_,"[u'MaharashtraCM', u'Maharashtra', u'Maharasht...",326
4,2019-11-26 09:45:40,BJP’s - One Night stand doesn’t last long #Ma...,MedhiMahesh,"[u'MahaThriller', u'MahaMLAParade', u'Maharash...",5


In [4]:
sentiments = pd.DataFrame(df.tweet_text)


In [5]:
import re

# function to remove user handles
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt  

# remove user handles
sentiments['clean_text'] = sentiments['tweet_text'].apply(lambda row:remove_pattern(str(row), "@[\w]*"))

In [6]:
# first five observations
sentiments.head()

,tweet_text,clean_text
0,So as per mananiya bhavi mukhyamantri Aditya T...,So as per mananiya bhavi mukhyamantri Aditya T...
1,@Dev_Fadnavis Sir these 4 days you made #Udha...,Sir these 4 days you made #UdhavThackeray a...
2,@PoulomiMSaha @sanjayraut @AjitPawarSpeaks @Of...,Which SEAT will fight ELECTIONS 🗳️🇮🇳🆒🆗🤙 ...
3,I wanna see @AUThackeray as our Education Mini...,I wanna see as our Education Minister! He was...
4,BJP’s - One Night stand doesn’t last long #Ma...,BJP’s - One Night stand doesn’t last long #Ma...


In [7]:
from nltk.corpus import stopwords

import nltk
from string import punctuation
from nltk.stem.porter import *
stop_words = list(set(stopwords.words('english')))+list(punctuation)+['``', "'s", "...",".", "n't","*"]

# tokenize
sentiments['tokenized_text'] = [nltk.word_tokenize(x) for x in sentiments['clean_text']]

# stopword removal
sentiments['tokenized_text'] = sentiments['tokenized_text'].apply(lambda row: [word for word in row if word not in stop_words])

# stemming words
stemmer = PorterStemmer()
sentiments['tokenized_text'] = sentiments['tokenized_text'].apply(lambda x: [stemmer.stem(i) for i in x])
sentiments['tokenized_text'] = sentiments['tokenized_text'].apply(lambda x: ' '.join(x))

sentiments.head()

,tweet_text,clean_text,tokenized_text
0,So as per mananiya bhavi mukhyamantri Aditya T...,So as per mananiya bhavi mukhyamantri Aditya T...,So per mananiya bhavi mukhyamantri aditya thac...
1,@Dev_Fadnavis Sir these 4 days you made #Udha...,Sir these 4 days you made #UdhavThackeray a...,sir 4 day made udhavthackeray adityathackeray ...
2,@PoulomiMSaha @sanjayraut @AjitPawarSpeaks @Of...,Which SEAT will fight ELECTIONS 🗳️🇮🇳🆒🆗🤙 ...,which seat fight elect 🗳️🇮🇳🆒🆗🤙 uddhavthackeray...
3,I wanna see @AUThackeray as our Education Mini...,I wanna see as our Education Minister! He was...,I wan na see educ minist He one student got in...
4,BJP’s - One Night stand doesn’t last long #Ma...,BJP’s - One Night stand doesn’t last long #Ma...,bjp ’ one night stand ’ last long mahathril ma...


In [8]:
sentiments = sentiments.dropna()

In [9]:
sentiments.isnull().sum()

tweet_text        0
clean_text        0
tokenized_text    0
dtype: int64

In [10]:
sentiments = sentiments.drop_duplicates()

In [11]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [12]:
# list to store polarities
tb_polarity = []

# loop over tweets
for sentence in sentiments['tokenized_text']:
    temp = TextBlob(sentence)
    tb_polarity.append(temp.sentiment[0])



In [16]:
txtblb_sentiment = []
for i in tb_polarity:
    if i>0:
        txtblb_sentiment.append('positive')
    elif i==0:
        txtblb_sentiment.append('neutral')
    else:
        txtblb_sentiment.append('negative')

In [18]:
sentiments['tb_polarity'] = txtblb_sentiment

In [19]:
sentiments.head()

,tweet_text,clean_text,tokenized_text,tb_polarity
0,So as per mananiya bhavi mukhyamantri Aditya T...,So as per mananiya bhavi mukhyamantri Aditya T...,So per mananiya bhavi mukhyamantri aditya thac...,neutral
1,@Dev_Fadnavis Sir these 4 days you made #Udha...,Sir these 4 days you made #UdhavThackeray a...,sir 4 day made udhavthackeray adityathackeray ...,negative
2,@PoulomiMSaha @sanjayraut @AjitPawarSpeaks @Of...,Which SEAT will fight ELECTIONS 🗳️🇮🇳🆒🆗🤙 ...,which seat fight elect 🗳️🇮🇳🆒🆗🤙 uddhavthackeray...,positive
3,I wanna see @AUThackeray as our Education Mini...,I wanna see as our Education Minister! He was...,I wan na see educ minist He one student got in...,negative
4,BJP’s - One Night stand doesn’t last long #Ma...,BJP’s - One Night stand doesn’t last long #Ma...,bjp ’ one night stand ’ last long mahathril ma...,negative


In [21]:
analyser = SentimentIntensityAnalyzer()
# list to store polarities
vader_polarity = []

# loop over tweets
for sentence in sentiments['tokenized_text']:
    vader_polarity.append(analyser.polarity_scores(sentence)['compound'])

In [25]:
vader_sentiment = []
for i in vader_polarity:
    if i>0:
        vader_sentiment.append('positive')
    elif i==0:
        vader_sentiment.append('neutral')
    else:
        vader_sentiment.append('negative')

In [28]:
sentiments = sentiments.drop('vader_sentiments',axis=1)

In [30]:
sentiments['vader_sentiments'] = vader_sentiment

In [31]:
sentiments

,tweet_text,clean_text,tokenized_text,tb_polarity,vader_sentiments
0,So as per mananiya bhavi mukhyamantri Aditya T...,So as per mananiya bhavi mukhyamantri Aditya T...,So per mananiya bhavi mukhyamantri aditya thac...,neutral,negative
1,@Dev_Fadnavis Sir these 4 days you made #Udha...,Sir these 4 days you made #UdhavThackeray a...,sir 4 day made udhavthackeray adityathackeray ...,negative,positive
2,@PoulomiMSaha @sanjayraut @AjitPawarSpeaks @Of...,Which SEAT will fight ELECTIONS 🗳️🇮🇳🆒🆗🤙 ...,which seat fight elect 🗳️🇮🇳🆒🆗🤙 uddhavthackeray...,positive,negative
3,I wanna see @AUThackeray as our Education Mini...,I wanna see as our Education Minister! He was...,I wan na see educ minist He one student got in...,negative,neutral
4,BJP’s - One Night stand doesn’t last long #Ma...,BJP’s - One Night stand doesn’t last long #Ma...,bjp ’ one night stand ’ last long mahathril ma...,negative,neutral
5,#MaharashtraCrisis shockingly #ShivaSena #Udha...,#MaharashtraCrisis shockingly #ShivaSena #Udha...,maharashtracrisi shockingli shivasena udhavtha...,neutral,positive
6,@maryashakil @Arunima24 What happens to @Shiv...,What happens to if it loses Floor Test? Wi...,what happen lose floor test will sit opposit c...,negative,positive
7,@rsprasad Why was Ajit Pawar was given clean c...,Why was Ajit Pawar was given clean chit by AC...,whi ajit pawar given clean chit acb bjp emerge...,positive,positive
8,@mieknathshinde #adityathackeray Good afternoo...,#adityathackeray Good afternoon how cum you s...,adityathackeray good afternoon cum sure 162 yo...,positive,positive
9,Supreme Court of India orders #MaharashtraGovt...,Supreme Court of India orders #MaharashtraGovt...,suprem court india order maharashtragovtform f...,neutral,neutral
